In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

d:\burtm\Visual_studio_code\conda_environments\GeneralPurposeML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
selected_model = 'trocr-small-stage1'

transform = u_transforms.get_transform(selected_model)
model = model_utils.get_model(name=selected_model, mode='truncated', pretrained=True, truncation='remove head')

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 1024,
  "decode

In [21]:
N_max=282
patches=True
input_filename='icdar_train_df_patches_20250515_164130.csv'
huggingface=True
pooling=False # if true in transformer mdoels use pooling, if false only the cls token

In [14]:
# Define loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ",device)
model = model.to(device)

Device is:  cuda


In [15]:
source_path="D:\\burtm\\Visual_studio_code\\PD_related_projects"
train_df = pd.read_csv(f"{source_path}\\outputs\\preprocessed_data\\{input_filename}")

In [17]:
i=0
output=compute_output(model, device, transform, train_df.iloc[i], huggingface, patches)

In [23]:
model.eval()

# Initialize a dictionary to store new feature columns
new_features = {}

for index,t in train_df.iterrows():
    if pooling:
        print("Pooling is not implemented yet")
        break
    else:
        output = compute_output(model, device, transform, t, huggingface, patches)[:,0,:]
    for i, value in enumerate(output.squeeze().tolist()):
        column_name = f"f{i+1}"
        if column_name not in new_features:
            new_features[column_name] = []
        new_features[column_name].append(value)
    if index % 100 == 0:
        print(f"Processed {index} images, out of {len(train_df)}")

    
# Add the new features to the DataFrame in one operation
new_features_df = pd.DataFrame(new_features)
train_df = pd.concat([train_df.reset_index(drop=True), new_features_df], axis=1)

Processed 0 images, out of 5640
Processed 100 images, out of 5640
Processed 200 images, out of 5640
Processed 300 images, out of 5640
Processed 400 images, out of 5640
Processed 500 images, out of 5640
Processed 600 images, out of 5640
Processed 700 images, out of 5640
Processed 800 images, out of 5640
Processed 900 images, out of 5640
Processed 1000 images, out of 5640
Processed 1100 images, out of 5640
Processed 1200 images, out of 5640
Processed 1300 images, out of 5640
Processed 1400 images, out of 5640
Processed 1500 images, out of 5640
Processed 1600 images, out of 5640
Processed 1700 images, out of 5640
Processed 1800 images, out of 5640
Processed 1900 images, out of 5640
Processed 2000 images, out of 5640
Processed 2100 images, out of 5640
Processed 2200 images, out of 5640
Processed 2300 images, out of 5640
Processed 2400 images, out of 5640
Processed 2500 images, out of 5640
Processed 2600 images, out of 5640
Processed 2700 images, out of 5640
Processed 2800 images, out of 56

In [24]:
train_df.head()

,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f375,f376,f377,f378,f379,f380,f381,f382,f383,f384
0,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,0,0,493,493,...,0.622814,0.652870,-0.036768,-0.072947,1.055691,-0.420997,0.843602,-0.768733,0.928809,-0.653204
1,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,1,493,493,986,...,0.466590,0.390466,-0.063519,-0.098352,0.813783,-0.317575,0.831497,-0.474965,0.726222,-0.933849
2,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,2,1479,493,1972,...,0.492431,0.539464,0.355833,-0.734787,1.332562,-0.239772,0.353327,-0.422959,0.952368,-0.632744
3,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,3,986,493,1479,...,0.411978,0.623444,0.031737,0.048326,0.664006,-0.226131,0.853005,-0.710104,1.464373,-0.562862
4,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,4,1972,493,2465,...,0.963354,0.558444,-0.039232,0.150385,1.289550,-0.446754,0.663165,-0.859034,1.164699,-0.655603


In [ ]:
'''from utils.dataframes import CustomImageDataset, CustomPatchDataset
from torch.utils.data import Dataset,DataLoader, random_split
source_path="D:\\burtm\\Visual_studio_code\\PD_related_projects"
train_df = pd.read_csv(f"{source_path}\\outputs\\preprocessed_data\\{input_filename}")
model.eval()
dataset = CustomPatchDataset(train_df[train_df['writer']<=N_max] ,
                                        label_column='male', transform=transform, huggingface=huggingface)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
new_features = {}

for idx, batch in enumerate(dataloader):
    patch = batch['image'].to(device)
    output = model(pixel_values=patch)
    cls_token_output = output.last_hidden_state[:, 0, :]  # Extract the CLS token
    for i, value in enumerate(cls_token_output.squeeze().tolist()):
        column_name = f"f{i+1}"
        if column_name not in new_features:
            new_features[column_name] = []
        new_features[column_name].append(value)
    if idx % 100 == 0:
        print(f"Processed {idx * batch_size} images, out of {len(dataset)}")

# Convert the dictionary to a DataFrame
new_features_df = pd.DataFrame(new_features)'''

Processed 0 images, out of 5640



KeyboardInterrupt



In [25]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = os.path.join(source_path, "outputs", "preprocessed_data")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

output_file = os.path.join(output_dir, f"icdar_EXTRACTED_train_df_{selected_model}_{timestamp}.csv")
train_df.to_csv(output_file, index=False)

print(f"Dataframe saved to {output_file}")

Dataframe saved to D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv


In [28]:
# Example usage:
LOG_FILE = output_dir+"\\file_metadata_log.json"
print(f"Log file path: {LOG_FILE}")
print(f"Output file path: {output_file}")
file_IO.add_or_update_file(
    output_file, LOG_FILE,
    custom_metadata={
        #"seed": seed,
        "source_file": input_filename,
        "model": selected_model,
        "pooling": pooling,
        "description": '''I am performing inference with trocr-small-stage1 on the patches dataset and saving the output features as columns of the dataframe
        the columns are named f1, f2, .., f384''' 
    }
)

Log file path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\file_metadata_log.json
Output file path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv
Updated log for icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv


In [29]:
file_IO.read_metadata(
    output_file,
    log_path=LOG_FILE
)

Metadata for icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv:
full_path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv
size_bytes: 43379066
created: 2025-05-16T12:25:28.589676
modified: 2025-05-16T12:25:33.533804
accessed: 2025-05-16T12:25:33.533804
source_file: icdar_train_df_patches_20250515_164130.csv
model: trocr-small-stage1
pooling: False
description: I am performing inference with trocr-small-stage1 on the patches dataset and saving the output features as columns of the dataframe
        the columns are named f1, f2, .., f384


In [15]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path=source_path+f"\\outputs\\preprocessed_data\\icdar_EXTRACTED_trocr_stage_1_10blocks_{timestamp}.csv"
train_df.to_csv(file_path, index=False)
print(f"File saved: {file_path}")

File saved: D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\icdar_EXTRACTED_trocr_stage_1_10blocks_20250509_153553.csv


# easy access

In [27]:
def reload_modules():
    import importlib
    import utils.data_loading as data_loading
    import utils.visualization as visualization
    import utils.dataframes as dataframes
    import utils.utils_transforms as u_transforms
    import utils.training_utils as training_utils
    import utils.model_utils as model_utils
    import utils.file_IO as file_IO
    
    importlib.reload(file_IO)
    importlib.reload(data_loading)
    importlib.reload(visualization)
    importlib.reload(dataframes)
    importlib.reload(u_transforms)
    importlib.reload(model_utils)
    importlib.reload(training_utils)

    return data_loading, visualization, dataframes, u_transforms, training_utils, model_utils, file_IO
data_loading, visualization, dataframes, u_transforms, training_utils, model_utils, file_IO = reload_modules()

In [16]:
def compute_output(model, device, transform, t, huggingface, patches):
    image_file = t['file_name']
    image = Image.open(image_file).convert("RGB")
    if patches:
        x1 = t['x']
        y1 = t['y']
        x2 = t['x2']
        y2 = t['y2']
        patch = image.crop((x1, y1, x2, y2))
    else:
        patch = image.copy()
    if huggingface:
        # the transform is actually an huggingface processor in this case
        inputs = transform(images=patch, return_tensors="pt")
        # Remove batch dimension from inputs
        patch = inputs['pixel_values'].squeeze()
    else:
        patch = transform(patch)
    patch = patch.to(device)
    output = model(patch.unsqueeze(0))
    return output

In [10]:
def show_image(transform, t, huggingface, patches):
    image_file = t['file_name']
    image = Image.open(image_file).convert("RGB")
    if patches:
        x1 = t['x']
        y1 = t['y']
        x2 = t['x2']
        y2 = t['y2']
        patch = image.crop((x1, y1, x2, y2))
    else:
        patch = image.copy()

    # Show original image/patch
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(patch)
    plt.title("Original")
    plt.axis('off')

    if huggingface:
        inputs = transform(images=patch, return_tensors="pt")
        transformed_patch = inputs['pixel_values'].squeeze()
        # Convert tensor to numpy for visualization
    else:
        transformed_patch = transform(patch)
    
    img_np = transformed_patch.permute(1, 2, 0).cpu().numpy()
    # Normalize if needed
    img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())

    plt.subplot(1, 2, 2)
    plt.imshow(img_np)
    plt.title("Transformed")
    plt.axis('off')
    plt.show()